## Basic PWS data processing using PWSpy-qc 
___


<img src="./pics/title.png" alt="drawing" width="8000"/>

**Jochen Seidel**, Abbas El Hachem, Micha Eisele, András Bárdossy
___
University of Stuttgart

## How does a (Netatmo) PWS work? 

<img src="./pics/PWS.png" alt="drawing" width="800"/>

## Why is it not advisable to use raw PWS data? Some typical (Netatmo) PWS errors:


### Many PWS are not installed/setup according to WMO standards

<img src="./pics/PWS_setup.png" alt="drawing" width="800"/>

### Incorrect information about the location

<img src="./pics/PWS_movements.png" alt="PWS on the move" width="800"/>

### Incorrect information about the location - "hot spots"

<img src="./pics/PWS_hotspots.png" alt="PWS on the move" width="1000"/>

### Data gaps and peaks

<img src="./pics/PWS_gaps.png" alt="PWS on the move" width="1000"/>

## The Bottom Line
### Due to the numerous potential errors sources the data from  Netatmo-PWS have to be QC and filtered! 

Overview of the three QC algorithms in [El Hachem et al. 2023](https://hess.copernicus.org/preprints/hess-2023-195/)

## 1. Loading PWS and auxilliary data (primary stations)

**Goal**: Load PWS and auxilliary data stored as csv as an `pandas.DataFrame` object and visualize the data

### 1.0 Import packages and adjust settings

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import xarray as xr
xr.set_options(display_style='text'); # Show xarray.Dataset representation as text

#import pycomlink as pycml

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 10}) # Set plot font size to 16pt

import numpy as np
import time
from scipy.spatial import cKDTree
from pykrige import OrdinaryKriging as OKpy
import tqdm
import os
import pandas as pd
import matplotlib as mpl

# OPENSENSE Sandbox tools 
# ToDo: Set link from local to github
import PWSpyqcFunctions as pyqc
import pycomlink as pycml

# Relative path to the data directory
main_dir = r".././data/pws/"


`PWSpy-qc` specific settings 

In [ ]:
# maximun distance (meters) for which the indicator correlation is calculated
max_distance = 50000

# Precentile threshold for indicator correlation 
prob=0.99

# todo: chekc if and where this is needed
show_plot = True

# minimum number of records (in hours) with values (here 2 months)
min_req_ppt_vals = 2*30*24 

## 1.1 Load the csv data files

In [ ]:
# Automatic weather satation data from KNMI Netherlands
path_primary_network = os.path.join(main_dir, r"data_AWS_stns_data.csv")
path_primary_metadata = os.path.join(main_dir, r"data_AWS_stns_coords.csv")   

# Data from 20 randomly chose radar grid cells as "primary stations 2" for the Amsterdam area
path_primary_network2 = os.path.join(main_dir, r"data_Radar_grid_cell_vals.csv")
path_primary_metadata2 = os.path.join(main_dir, r"data_selected_radar_grid_lonlat.csv") 


# PWS hourly data ()
path_pws_data = os.path.join(main_dir, r"data_AMS_PWS_hourly_shifted.csv")   
path_pws_metadata = os.path.join(main_dir, r"data_AMS_metadata.csv")

**&#8594;** each datasets consists of a metafile with coordinates and a data file with stations as columns and timestamps as rows (c.f. OPENSENSE White Paper)

**&#8594;** the Netatmo PWS data set used here is published by de [Vos (2019)](https://data.4tu.nl/articles/_/12703250/1)

**&#8594;** `pyqc.read_pcp_csv_file` and `read_metadata_csv_file` in the following code are functions based on `pandas.DataFrame` 

If you are familiar with python functions you can have a look at the file **PWSpyqcFunctions.py**

In [ ]:
# read primary network 1
print('Reading first primary network data')
# pyqc.read_pcp_csv_file has 'latin-1' encoding in PWSpyqcFunctions.py
in_primary_pcp = pyqc.read_pcp_csv_file(path_to_file=path_primary_network,
                           sep_type=';',
                           index_col=0)

# pyqc uses EPSG:25832, for Netherlands EPSG:28531 is used!
# Reads coordinate file and additionally coverts Lat/Lon to UTM 32
df_prim_coords, prim_coords_xy = pyqc.read_metadata_csv_file(path_primary_metadata,
                                                           sep_type=';', index_col=0,)
df_prim_coords.index = in_primary_pcp.columns
# read primary network 2
print('Reading secondary primary network data')
in_primary_pcp_2 = pyqc.read_pcp_csv_file(path_to_file=path_primary_network2,
                           sep_type=';',
                           index_col=0)

df_prim_coords_2, prim_coords_xy_2 = pyqc.read_metadata_csv_file(
            path_primary_metadata2,
            sep_type=',', index_col=0,)
df_prim_coords_2.index = in_primary_pcp_2.columns

# read pws data
print('Reading PWS data')
df_pws_pcp_hourly = pyqc.read_pcp_csv_file(
        path_to_file=path_pws_data,
                           sep_type=';',
                           index_col=0)
    
df_pws_coords, pws_coords_xy = pyqc.read_metadata_csv_file(
        path_to_file=path_pws_metadata,
                           sep_type=',',
                           index_col=0)
df_pws_coords.index = df_pws_pcp_hourly.columns

## 1.2 How do the data look like?

In [ ]:
# This shows the PWS data itself as described above
df_pws_pcp_hourly

In [ ]:
df_pws_pcp_hourly.describe()

### Questions:

* What does count mean?
* Why are all the 75% percentiles 0.0?
* How can you display other percentiles, e.g. 0.99?

## 1.3 Plotting the data

### 1.3.1 Location of the stations

In [ ]:
print('Plotting locations of all data')

pyqc.plot_pws_prim_netz_loc(lon_pws=df_pws_coords.lon,
                       lat_pws=df_pws_coords.lat,
                      lon_prim1=df_prim_coords.lon,
                       lat_prim1=df_prim_coords.lat,
                      lon_prim2=df_prim_coords_2.lon,
                       lat_prim2=df_prim_coords_2.lat)

    


In [ ]:
print('Plotting locations of data in and around Amsterdam')

pyqc.plot_pws_prim_netz_loc_AMS(lon_pws=df_pws_coords.lon,
                       lat_pws=df_pws_coords.lat,
                      lon_prim1=df_prim_coords.lon,
                       lat_prim1=df_prim_coords.lat,
                      lon_prim2=df_prim_coords_2.lon,
                       lat_prim2=df_prim_coords_2.lat)

### 1.3.2 Plotting the PWS data

In [ ]:
# Plotting all data in the DataFrame (combined plot with 134 stations does not really make sense...)
df_pws_pcp_hourly.plot(legend=False, ylabel='mm')

This does not really make sense with 134 stations.

+ What is the first thing we can observe here?

### 1.4 Let's plot some individual stations

In [ ]:
# PWS station ams18
df_pws_pcp_hourly['ams18'].plot(ylabel='mm', title='PWS ams18')


**&#8594;** This PWS hat a more or less continuous record (last part is missing) and looks ok at first sight

In [ ]:
# PWS station ams46
df_pws_pcp_hourly['ams46'].plot(ylabel='mm', title='PWS ams46')

**&#8594;** This PWS has either low values or missing values, apparently no 0mm occur. Furthermore, this series is rather short.

In [ ]:
# PWS station ams105
df_pws_pcp_hourly['ams105'].plot(ylabel='mm', title='PWS ams105')

**&#8594;** This PWS has unrealstic data values and a large gap until September 2017. However, the data from September 2017 onwards seems ok.

### Exercises: 

+ Plot the data for an arbritatry PWS from the dataframe


+ Plot the data for the 'ams105' station from Sept 2017 onwards

**Hint:** For `pandas dataframes` this to simiar to slicing in `xarray`, i.e. datetimes as strings in square bracktes separated by a colon `['yyyy-mm-dd':'yyyy-mm-dd']`


+ Remove this station's data before Sep 2017 (maybe make a copy of the dataframe first...)

**Hint:** You need to select a time interval (like slicing above) for a station and set these values to NaN 

In [ ]:
# Write code here:







**Caution:** Running the following cell will load the solution!

In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load ../solutions/PWSpyqc_1_solution.py


### Analysis of missing values (NaN)

In [ ]:
# Sorted number and %-ages of missing values per station
pyqc.missing_values_table(df_pws_pcp_hourly)

In [ ]:
# Plot a histogram with numbers of NaN per station
plt.hist(df_pws_pcp_hourly.isna().sum(), bins = np.arange(1000,19000,1000))
plt.ylabel('Number of PWS')
plt.xlabel('Number of NaN')

### 1.4. Summary Part 1

+ Raw (Netatmo) PWS data are often erroneous and need quality control before used in applications
+ Simple analyis of common errors
+ csv format is also possible to use for OS-data, however netCDF has several advantages

## 2. First PWSpy-QC step: Indicator correlation based filter
The fist QC-filter of PWS-pyqc is based on indicator correlation patterns of the primary network (cf. [Bárdossy et al. 2021](https://doi.org/10.5194/hess-25-583-2021)). First, the data of the primary network are converted to 1 and 0 based on a threshhold, in this case the 99%-percentile. All values above this percentile are 1, all below 0. This indicator correaltion of the primay stations has a spatial structure, i.e. the correlation decreases with increasing distance

In [ ]:
# Plot indicator correlation values different networks
# calculate indicator correlation primary network
print('Calculating indicator correlation')
dist_prim, corr_prim = pyqc.calc_indic_corr_all_stns(
                             coords_stns_xy=prim_coords_xy,
                             pcp_vals=in_primary_pcp.values, prob=0.99)

# calculate indicator correlation second primary network
dist_prim2, corr_prim2 = pyqc.calc_indic_corr_all_stns(
    coords_stns_xy=prim_coords_xy_2,
    pcp_vals=in_primary_pcp_2.values, prob=0.99)

# claculate indicator correlation PWS-PWS
dist_pws, corr_pws = pyqc.calc_indic_corr_all_stns(
                    coords_stns_xy=pws_coords_xy,
                         pcp_vals=df_pws_pcp_hourly.values)

# plot indicator correlations
pyqc.plot_indic_coor(dist_prim=dist_prim,
                corr_prim=corr_prim,
                dist_pws=dist_pws,
               corr_pws=corr_pws)

We can see that there is a large gap up to 20 km in the primary network, hence there's information about the spatial structure missing. This is the reason why the data from the KMNI gauge adjusted radar product are used in addtion

### Excercise: 
+ Plot the dependece structure with the secondary primary network (i.e. data from radar pixels)

**Hint:** Click on the function in the cell below and use shift Tab shows the syntax that the functions expect. Add the corresponding parameters

In [ ]:
pyqc.plot_indic_coor()

In [ ]:
#write your code here:







**Caution:** Running the following cell will load the solution!

In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load ../solutions/PWSpyqc_2_solution.py


As can be seen in the plot above, the primary stations have a certain dependence sturcture over distance. Most of the PWS as well, but there are also some PWS which do not follow this structure. When the indicator corrleation of a PWS next to a primary station is low, then it is likley that the PWS data are faulty. The first QC-filter pf PWS-pyqc now removes all PWS which do not show indicator correlations similiar to that of the reference data (primary stations).

### Applying the Indicator correlation based filter¶

In [ ]:
# apply indicator filter (using primary stations 2)

stn_in = pyqc.indicator_filter(xy_net=pws_coords_xy,
                 prc_net=df_pws_pcp_hourly,
                 xy_dwd=prim_coords_xy_2,
                 prc_dwd=in_primary_pcp_2,
                     prob=0.99, max_distance=max_distance,
                     min_req_ppt_vals=2*24*30, show_plot=True,
                     fn_figure='Indicator Correlation',
                     save_folder=None,
                    tolerance=.99)

# what data type is stn_in
# Was ist tolerance? (1 muss größer als min) Parameter zum aus probieren

In [ ]:
# list the accepted stations
pws_ids_accepted = df_pws_pcp_hourly.columns[np.where(stn_in==True)]
pws_ids_accepted

This filter compares the PWS with all primary stations within the set `max_distance` (see above), i.e. if the indicator correlation low to the next primary stations the PWS might still be accepted if it fit's to a primary station further away. `threshold` -> how much lower an IC compared to the primary network is still accepted 

### Excercise:

+ Change the `max_distance` and `threshold` parameters and see how these influence the results

In [ ]:
# copy code for indicator filter here and change the threshold for the indicator correlation:







**Question:** Why do you have to decrease the threshold for increasing temporal aggregation, e.g. for daily data?

## 3. Bias Correction

PWS can over- or undererstimate the precepitation compared to a professional refernce station (panel b) in figure below). The reasons herefore are mayfold (c.f. Introduction). The goal is to correct the PWS distribution function usind the distribution function of neighbouring primary network stations. For more details see [Bárdossy et al. (2021)](https://doi.org/10.5194/hess-25-583-2021)

<img src="./pics/bias.png" alt="drawing" width="800"/>


### Basic Scheme of the PWS-pyqc Bias Correction


<img src="./pics/bias_correction.png" alt="drawing" width="400"/>

In [ ]:
df_pws_bias_corrected = pyqc.bias_corr_pws_cdf(df_pws_raw=df_pws_pcp_hourly,
                        pws_ids_accepted=pws_ids_accepted,
                        df_pws_coords=df_pws_coords,                            
                        prim_coords_xy_2=prim_coords_xy_2,
                        in_primary_pcp_2=in_primary_pcp_2,
                        df_prim_coords_2=df_prim_coords_2,
                        nstns_correct=2,#len(pws_ids_accepted), #2,
                        plot_figures=True,
                        ppt_min_thr_0_vals=0.1,  # below it all values get p0/2
                        min_qt_to_correct=0.9,)

The bias correction is rather slow, that's why this is only shown for 2 stations. In both cases the PWS underestimates the precipitation and thus the values are corrected towards higher values.



### Excercise:

+ Plot the time series for PWS ams1 before and after the bias correction

  **Hint:** The bias correction function returns a new dataframe called `df_pws_bias_corrected`

In [ ]:
# Write code here:







**Caution:** Running the following cell will load the solution!

In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load ../solutions/PWSpyqc_3_solution.py

A different faster bias correction function is used here to correct all stations. This data set will be used for interpolation in the next section

In [ ]:
new_prc = pyqc.bias_correct(xy_net=pws_coords_xy,
                       prc_net=df_pws_pcp_hourly.values,
                       xy_prim=prim_coords_xy_2,
                       prc_prim=in_primary_pcp_2.values,
                           stn_in_bool=stn_in)

df_corrected = pd.DataFrame(index=df_pws_pcp_hourly.index,
                              data=new_prc,
                              columns =df_pws_pcp_hourly.columns)

# Für alle stationen?

## 4. Interpolate rainfall map from filtered and bias corrected PWS for a timestep

The goal is to correct the PWS distribution function usind the distribution function of neighboring primary network stations. For more details see [Bárdossy et al. (2021)](https://doi.org/10.5194/hess-25-583-2021)

We will also use the IDW interpolation routine from `pycomlink`

In the cell below which sets up the IDW Interpolator in the line

`pcp_all=df_corrected.iloc[18210, stn_in].values`

the number `18210` refers to the index of a timestep where most PWS have recorded precipitation. If you , you can try to find all timesteps where e.g. more than 60 PWS have recorded precipitation and choose a corresponding index from there. Otherwise you can just continue using the index `18210`


In [ ]:
# write your code for finding time steps where most PWS have recorded precipitation








**Caution:** Running the following cell will load the solution!

In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load ../solutions/PWSpyqc_41_solution.py

In [ ]:
# Set up the IDW interpolator
idw_interpolator = pycml.spatial.interpolator.IdwKdtreeInterpolator(
    nnear=15, 
    p=2, 
    exclude_nan=True, 
    max_distance=0.3)

#Create coordinates and data for interpolating values for a given timestamp/index
lon_pws=df_pws_coords.loc[pws_ids_accepted]
lat_pws=df_pws_coords.loc[pws_ids_accepted]
pcp_all=df_corrected.iloc[18210, stn_in].values

#Create indices for valid stations for the specific time step
idx=np.where(pcp_all>= 0)

R_grid = idw_interpolator(
    x=lon_pws.lon.values[idx], 
    y=lat_pws.lat.values[idx], 
    z=pcp_all[idx], 
    resolution=0.01,)

In [ ]:
# Plot the interpolated map and the locations of the accepted PWS
bounds = np.arange(0, 8, 0.5)
norm = mpl.colors.BoundaryNorm(boundaries=bounds, ncolors=256, extend='both')
fig, ax = plt.subplots(figsize=(8, 6))
pc = plt.pcolormesh(
    idw_interpolator.xgrid, 
    idw_interpolator.ygrid, 
    R_grid, 
    shading='nearest', 
    cmap='Blues',
    norm=norm,
)
plt.scatter(lon_pws.lon,lat_pws.lat, marker='o', color='r', alpha=.6)
fig.colorbar(pc, label='Rainfall sum in mm');
plt.show()


### Excercise:

+ Plot the original raw data for the same timestep and compare the results. Which parts int he codse do you have to change?

**Hint:** copy & paste both cells from above and replace the corresponding variables under

`#Create coordinates and data for interpolating values for a given timestamp/index` 


In [ ]:
# write/copy/change code here:





**Caution:** Running the following cell will load the solution!

In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load ../solutions/PWSpyqc_42a_solution.py

Finally, we want to plot the difference between these two maps. Create a new grid showing the differences between the filtered and bias corrected PWS and the Raw data.
**Hint:** As there fewer PWS in the filtered and bias corrected data than in the raw PWSdata set, the shapes of the Grids are different. This requires a work-around. 


In [ ]:
# Copy/write/change your code here:











**Caution:** Running the following cell will load the solution!

In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load ../solutions/PWSpyqc_42b_solution.py